**Importación de librerías y cargado de datos**

In [1]:
import pandas as pd
import numpy as np

print(F"Importación ok. Versión de Pandas instalada = {pd.__version__}")
print(F"Importación ok. Versión de NumPy instalada = {np.__version__}")

Importación ok. Versión de Pandas instalada = 2.2.2
Importación ok. Versión de NumPy instalada = 1.26.4


In [2]:
df = pd.read_csv("datos_ejemplo.csv")
df

,Región,Producto,Mes,Ventas,Unidades
0,Centro,A,Enero,305,48
1,Norte,A,Marzo,180,15
2,Centro,B,Marzo,661,8
3,Centro,B,Enero,971,14
4,Norte,A,Enero,487,23
5,Norte,A,Marzo,101,40
6,Centro,A,Febrero,489,21
7,Sur,C,Enero,665,16
8,Centro,C,Febrero,205,45
9,Centro,C,Febrero,871,18


In [3]:
print("📊 DATOS ORIGINALES")
print(df.head())
print(f"\nShape: {df.shape}")

📊 DATOS ORIGINALES
   Región Producto    Mes  Ventas  Unidades
0  Centro        A  Enero     305        48
1   Norte        A  Marzo     180        15
2  Centro        B  Marzo     661         8
3  Centro        B  Enero     971        14
4   Norte        A  Enero     487        23

Shape: (20, 5)


In [4]:
# Crear índice jerárquico con Región y Producto
df_jerarquico = df.set_index(['Región', 'Producto']).sort_index()  # ✅ Ordenar el índice
print("DataFrame con índice jerárquico (Región, Producto):")
df_jerarquico.head(8)

DataFrame con índice jerárquico (Región, Producto):


Mes  Ventas  Unidades
Región Producto                           
Centro A           Enero     305        48
       A         Febrero     489        21
       B           Marzo     661         8
       B           Enero     971        14
       B         Febrero     921        47
       B           Enero     501        25
       C         Febrero     205        45
       C         Febrero     871        18

In [5]:
# Acceso a datos específicos
print("\n📍 Acceso a datos del Centro:")
df_jerarquico.loc['Centro']


📍 Acceso a datos del Centro:


,Mes,Ventas,Unidades
Producto,,,
A,Enero,305,48
A,Febrero,489,21
B,Marzo,661,8
B,Enero,971,14
B,Febrero,921,47
B,Enero,501,25
C,Febrero,205,45
C,Febrero,871,18
C,Enero,576,24


In [6]:
print("\n📍 Acceso específico - Centro, Producto A:")
df_jerarquico.loc[('Centro', 'A')]


📍 Acceso específico - Centro, Producto A:


Mes  Ventas  Unidades
Región Producto                           
Centro A           Enero     305        48
       A         Febrero     489        21

In [7]:
# Agrupamiento básico por región
print("💰 Ventas totales por región:")
ventas_region = df.groupby('Región')['Ventas'].sum()
print(ventas_region)

💰 Ventas totales por región:
Región
Centro    5500
Norte     2225
Sur       3387
Name: Ventas, dtype: int64


In [8]:
print("\n📦 Estadísticas de unidades por producto:")
stats_producto = df.groupby('Producto')['Unidades'].agg(['mean', 'sum', 'count', 'min', 'max'])
print(stats_producto)


📦 Estadísticas de unidades por producto:
               mean  sum  count  min  max
Producto                                 
A         32.000000  192      6   15   48
B         26.833333  161      6    8   47
C         24.125000  193      8    1   45


In [10]:
print("\n📅 Ventas promedio por mes:")
ventas_mes = df.groupby('Mes')['Ventas'].mean().round(2)
print(ventas_mes)


📅 Ventas promedio por mes:
Mes
Enero      543.71
Febrero    686.17
Marzo      455.57
Name: Ventas, dtype: float64


In [11]:
# Aplicar múltiples funciones a la vez
print("📊 Análisis completo por región:")
analisis_region = df.groupby('Región').agg({
    'Ventas': ['sum', 'mean', 'count'],
    'Unidades': ['sum', 'max', 'min']
}).round(2)
print(analisis_region)

📊 Análisis completo por región:
       Ventas               Unidades        
          sum    mean count      sum max min
Región                                      
Centro   5500  611.11     9      250  48   8
Norte    2225  445.00     5      145  41  15
Sur      3387  564.50     6      151  45   1


In [12]:
print("\n📈 Resumen por producto:")
resumen_producto = df.groupby('Producto').agg({
    'Ventas': ['sum', 'mean'],
    'Unidades': 'sum'
}).round(2)
print(resumen_producto)


📈 Resumen por producto:
         Ventas         Unidades
            sum    mean      sum
Producto                        
A          1931  321.83      192
B          4511  751.83      161
C          4670  583.75      193


**Pivoteo de datos**

In [13]:
# Crear tabla pivote: Regiones vs Productos (suma de ventas)
print("🔄 Tabla pivote - Ventas por Región y Producto:")
pivot_ventas = df.groupby(['Región', 'Producto'])['Ventas'].sum().unstack(fill_value=0)
print(pivot_ventas)

🔄 Tabla pivote - Ventas por Región y Producto:
Producto    A     B     C
Región                   
Centro    794  3054  1652
Norte     768  1457     0
Sur       369     0  3018


In [14]:
print("\n🔄 Tabla pivote - Unidades por Mes y Región:")
pivot_unidades = df.groupby(['Mes', 'Región'])['Unidades'].sum().unstack(fill_value=0)
print(pivot_unidades)


🔄 Tabla pivote - Unidades por Mes y Región:
Región   Centro  Norte  Sur
Mes                        
Enero       111     23   31
Febrero     131     26   45
Marzo         8     96   75


**Despivoteo de datos**

In [15]:
# Despivotear la tabla anterior
print("🔄 Despivoteo de la tabla de ventas:")
df_melted = pivot_ventas.reset_index().melt(
    id_vars=['Región'],
    var_name='Producto',
    value_name='Ventas_Total'
)
print(df_melted[df_melted['Ventas_Total'] > 0])  # Solo mostrar valores > 0

🔄 Despivoteo de la tabla de ventas:
   Región Producto  Ventas_Total
0  Centro        A           794
1   Norte        A           768
2     Sur        A           369
3  Centro        B          3054
4   Norte        B          1457
6  Centro        C          1652
8     Sur        C          3018


In [16]:
print("\n🔄 Despivoteo completo del DataFrame original:")
df_melt_completo = df.melt(
    id_vars=['Región', 'Producto', 'Mes'],
    value_vars=['Ventas', 'Unidades'],
    var_name='Métrica',
    value_name='Valor'
)
print(df_melt_completo.head(8))


🔄 Despivoteo completo del DataFrame original:
   Región Producto      Mes Métrica  Valor
0  Centro        A    Enero  Ventas    305
1   Norte        A    Marzo  Ventas    180
2  Centro        B    Marzo  Ventas    661
3  Centro        B    Enero  Ventas    971
4   Norte        A    Enero  Ventas    487
5   Norte        A    Marzo  Ventas    101
6  Centro        A  Febrero  Ventas    489
7     Sur        C    Enero  Ventas    665


**Combinación de datos**

In [17]:
# Crear DataFrames adicionales para demostrar combinaciones
df1 = df[df['Región'] == 'Centro'][['Región', 'Producto', 'Ventas']].head(3)
df2 = df[df['Región'] == 'Norte'][['Región', 'Producto', 'Ventas']].head(3)

print("DataFrame 1 (Centro):")
print(df1)
print("\nDataFrame 2 (Norte):")
print(df2)

DataFrame 1 (Centro):
   Región Producto  Ventas
0  Centro        A     305
2  Centro        B     661
3  Centro        B     971

DataFrame 2 (Norte):
  Región Producto  Ventas
1  Norte        A     180
4  Norte        A     487
5  Norte        A     101


In [18]:
# CONCATENACIÓN
print("\nCONCATENACIÓN con concat():")
df_concat = pd.concat([df1, df2], ignore_index=True)
print(df_concat)


CONCATENACIÓN con concat():
   Región Producto  Ventas
0  Centro        A     305
1  Centro        B     661
2  Centro        B     971
3   Norte        A     180
4   Norte        A     487
5   Norte        A     101


In [19]:
# Crear DataFrame de información adicional para merge
info_productos = pd.DataFrame({
    'Producto': ['A', 'B', 'C'],
    'Categoría': ['Electrónicos', 'Ropa', 'Hogar'],
    'Precio_Unitario': [25.50, 80.00, 45.75]
})

print("DataFrame de información de productos:")
print(info_productos)

DataFrame de información de productos:
  Producto     Categoría  Precio_Unitario
0        A  Electrónicos            25.50
1        B          Ropa            80.00
2        C         Hogar            45.75


In [20]:
# MERGE - diferentes tipos de uniones
print("\nMERGE - Unión INNER:")
df_inner = df.merge(info_productos, on='Producto', how='inner')
print(df_inner[['Región', 'Producto', 'Ventas', 'Categoría', 'Precio_Unitario']].head(5))


MERGE - Unión INNER:
   Región Producto  Ventas     Categoría  Precio_Unitario
0  Centro        A     305  Electrónicos             25.5
1   Norte        A     180  Electrónicos             25.5
2  Centro        B     661          Ropa             80.0
3  Centro        B     971          Ropa             80.0
4   Norte        A     487  Electrónicos             25.5


In [21]:
print("\nMERGE - Unión LEFT:")
df_left = df.merge(info_productos, on='Producto', how='left')
print(f"Filas originales: {len(df)}, Filas después del merge: {len(df_left)}")


MERGE - Unión LEFT:
Filas originales: 20, Filas después del merge: 20


In [22]:
# Ejemplo práctico: Calcular ingresos totales
print("\nEJEMPLO PRÁCTICO - Cálculo de ingresos:")
df_ingresos = df.merge(info_productos, on='Producto', how='left')
df_ingresos['Ingresos_Calculados'] = df_ingresos['Unidades'] * df_ingresos['Precio_Unitario']
df_ingresos[['Región', 'Producto', 'Mes', 'Ventas', 'Unidades', 'Precio_Unitario', 'Ingresos_Calculados']].head()


EJEMPLO PRÁCTICO - Cálculo de ingresos:


,Región,Producto,Mes,Ventas,Unidades,Precio_Unitario,Ingresos_Calculados
0,Centro,A,Enero,305,48,25.5,1224.0
1,Norte,A,Marzo,180,15,25.5,382.5
2,Centro,B,Marzo,661,8,80.0,640.0
3,Centro,B,Enero,971,14,80.0,1120.0
4,Norte,A,Enero,487,23,25.5,586.5
